# Data Generation

Simulated data has been generated using the threebox model from [Kirchner 2019](https://hess.copernicus.org/articles/23/303/2019/). 


In this notebook, we explain the procedure we followed to generate the 6 benchmark datasets used in our paper.

## /!\ The Pyeto Python package should be downloaded from [here](https://github.com/woodcrafty/PyETo) and the path of its location on your laptop should be added using the command below:

In [ ]:
import sys
sys.path.append('{Path where you saved the Pyeto package}')

# 1. Loading the precipitation time series and computing potential evapotranspiration using daily (Tmax, Tmin and Tmean)


Note that you will need the (hourly) precipitation time series and the daily maximum, minimum and mean temperature at the site of interest. This data can be computed for example on the MeteoSwiss' API for catchment located in Switzerland.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
import pyeto
import GAMCR

def toYearFraction(date):
    """
    Allows to convert a date into a timeyear object (i.e. such as 2022.5 for summer in 2022).
    """
    def sinceEpoch(date): # returns seconds since epoch
        return (date.timestamp())
    s = sinceEpoch

    year = date.year
    startOfThisYear = datetime(year=year, month=1, day=1)
    startOfNextYear = datetime(year=year+1, month=1, day=1)

    yearElapsed = s(date) - s(startOfThisYear)
    yearDuration = s(startOfNextYear) - s(startOfThisYear)
    fraction = yearElapsed/yearDuration

    return date.year + fraction

# Defining the site and its latitude
site = 'BAS'
site2name = {'SIO':'Sion', 'LUG':'Lugano', 'PUY':'Pully', 'BAS':'Basel'}
ls_latitude = {'LUG': 46, 'SIO': 46.13, 'PUY': 46.31, 'BAS':47.32}


# Loading the precipitation data
df = pd.read_csv('./precipitation/order_122969_{0}_rre150h0_1_data.txt'.format(site), sep=';')
df['datetime'] = df['time'].apply(lambda x: datetime.strptime(str(x), '%Y%m%d%H'))
df['t'] = df['datetime'].apply(lambda x: toYearFraction(x))

df = df.replace('-', np.NaN)
df = df.rename(columns={"rre150h0": "p"})
df['p'] = df['p'].astype(float)

In [ ]:
# Making sure there is no gaps in the data
diffhour = (df['datetime'].sort_values().diff() > pd.to_timedelta('1 hour')).to_numpy()
idxs = np.where(diffhour>0.5)[0]
plt.plot(diffhour)

In [ ]:
# Loading daily temprature data
Tmax = pd.read_csv('./daily_temperature_min_max/order_123003_{0}_tre200dx_1_data.txt'.format(site), header=0, sep=';', engine='python')
Tmax['datetime'] = Tmax['time'].apply(lambda x: datetime.strptime(str(x), '%Y%m%d'))
Tmax['t'] = Tmax['datetime'].apply(lambda x: toYearFraction(x))
Tmax = Tmax.rename(columns={"tre200dx": "tmax"})
Tmax = Tmax.replace('-', np.NaN)

Tmin = pd.read_csv('./daily_temperature_min_max/order_123003_{0}_tre200dn_1_data.txt'.format(site), header=0, sep=';',  engine='python')
Tmin['datetime'] = Tmin['time'].apply(lambda x: datetime.strptime(str(x), '%Y%m%d'))
Tmin['t'] = Tmin['datetime'].apply(lambda x: toYearFraction(x))
Tmin = Tmin.rename(columns={"tre200dn": "tmin"})
Tmin = Tmin.replace('-', np.NaN)

Tmean = pd.read_csv('./daily_temperature_mean/order_122970_{0}_tre200d0_1_data.txt'.format(site), header=0, sep=';',  engine='python')
Tmean['datetime'] = Tmean['time'].apply(lambda x: datetime.strptime(str(x), '%Y%m%d'))
Tmean['t'] = Tmean['datetime'].apply(lambda x: toYearFraction(x))
Tmean = Tmean.rename(columns={"tre200d0": "tabs"})
Tmean = Tmean.replace('-', np.NaN)

data = Tmin
#data = pd.merge(data, Tmin, on="datetime", how="inner")
data = pd.merge(data, Tmax, on="datetime", how="inner")
data = pd.merge(data, Tmean, on="datetime", how="inner")

diffhour = (data['datetime'].sort_values().diff() > pd.to_timedelta('1 day')).to_numpy()
idxs = np.where(diffhour>0.5)[0]
plt.plot(diffhour)

In [ ]:
# Computing PET using the Hargreaves method implemented in the Pyeto Package
def get_PET_hargreaves(tmin,tmean,tmax,Date,latitude):
    lat = pyeto.deg2rad(float(latitude))  # Convert latitude to radians
    day_of_year = Date.timetuple().tm_yday
    sol_dec = pyeto.sol_dec(day_of_year)            # Solar declination
    sha = pyeto.sunset_hour_angle(lat, sol_dec)
    ird = pyeto.inv_rel_dist_earth_sun(day_of_year)
    et_rad = pyeto.et_rad(lat, sol_dec, sha, ird)   # Extraterrestrial radiation
    tmax = max(tmax,tmin)
    tmin = min(tmin,tmax)
    tmean = max(min(tmean,tmax),tmin)
    return pyeto.hargreaves(tmin, tmax, tmean, et_rad)

lst = np.arange(0,len(data),1)
PET = np.array(list(map(lambda t: get_PET_hargreaves(data.iloc[t]['tmin'], data.iloc[t]['tabs'], data.iloc[t]['tmax'], data['datetime'][t], ls_latitude[site]), lst) ))
data['pet'] = PET

In [ ]:
# We add the PET data to the dataframe with precipitation making sure that dates match
df = df.sort_values(by='datetime', axis=0)
data = data.sort_values(by='datetime', axis=0)
begindate = max(df.iloc[0]['datetime'], data.iloc[0]['datetime'])
enddate = min(df.iloc[-1]['datetime'], data.iloc[-1]['datetime'])
enddate = enddate-pd.to_timedelta('1 hour')
print(begindate, enddate)
df = df.loc[df['datetime'].apply(lambda x: (begindate<=x) and (enddate>=x))]
data = data.loc[data['datetime'].apply(lambda x: (begindate<=x) and (enddate>=x))]

In [ ]:
# We convert daily PET to hourly data
df_index = data[['datetime','pet']].set_index('datetime')
hours2daily = df['datetime'].apply(lambda x: df_index.loc[x.strftime('%Y-%m-%d')].values[0])
#hours2hourlypet = [data.loc[data['datetime']==day]['pet'].values[0]/24 for day in hours2days]
df['pet'] = hours2daily/24

df.fillna(0, inplace=True)
df['p'].isnull().sum()

In [ ]:
df = df.rename(columns={"t": "timeyear"})
df['date'] = df['timeyear'].apply(lambda x: GAMCR.fractional_year_to_datetime(x))

# We save the dataframe
df.to_csv('data_{0}.csv'.format(site2name[site]), index=False)

# 2. (Optional) Calibrating the parameters of the threebox model

When considering precipitation data from a real site, one might want to generate streamflow values that are realistic for this site. To do so, one can calibrate the model parameters of the threebox models using both precipitation and streamflow time series at the given site (or at a similar site for which data is available).

# 3. Running the threebox with to get the simulated streamflow time series

You have two possible options here:

- either you simply want to get the streamflow time series:
In this case you can use the Rscript `generate_streamflow.R` located in threeboxmodel/Rscripts.
- or you also want to compute the true transfer functions:
In this case you can use the Rscript `simulations_computing_tfs_flashy.R` located in threeboxmodel/Rscripts. This script will save the data related to the simulation on the complete precipitation time series in a file entitled `ref_flow_notflashy_fluxes.txt`. The Rscript then launched the simulator again by removing a single precipitation event. Note that you can specify the minimum precipitation intensity for which you want to compute the corresponding transfer function. By default, the threshold is set to 1 (mm/h). 
Once the Rscript is finished, you can use the python script `save_tfs.py` (also located in threeboxmodel/Rscripts) to aggregate and save the transfer functions of every precipitation events of interest in a single file.

Note that in both cases, you should modify the Rscript to: 
- change the path the file according to the location of the code on your laptop
- change if needed the model parameters
- change the name of the site considered

# 4. Visualizing the data

You can visualize the simulated data (and the computed transfer functions) by relying on the notebook `visualization_data.ipynb` located in the folder `notebooks`.

# 5. Making sure we have the right folder structure

To properly use the GAMCR package, you should have a folder for each site with the following structure:

- this folder should have name `site`
- in this folder, you should have the `data_{site}.txt` file saved
- GAMCR will save in this folder the different models that you will train for that site
- in this folder, two subfolders will be created and used by GAMCR. 
    * The first subfolder `data` will be created to save the preprocessed data when calling a `save_batch` type method
    * The second subfolder `results` will be created to save some statistics on the results of a trained model when calling the `compute_statistics` method

![Screen shot of an example of simulated data.](../../_static/simulated_data.png)